# Run PyTorchJob From Function

In this Notebook we are going to create [Kubeflow PyTorchJob](https://www.kubeflow.org/docs/components/training/pytorch/).

The PyTorchJob will run distributive training using [DistributedDataParallel strategy](https://pytorch.org/docs/stable/generated/torch.nn.parallel.DistributedDataParallel.html).

## Install Kubeflow Python SDKs

You need to install PyTorch packages and Kubeflow SDKs to run this Notebook.

In [ ]:
!pip install torch==2.1.2
!pip install torchvision==0.19.1

# TODO (andreyvelich): Change to release version when SDK with the new APIs is published.
!pip install git+https://github.com/kubeflow/training-operator.git#subdirectory=sdk/python

## Create Train Script for CNN Model

This is simple **Convolutional Neural Network (CNN)** model for recognizing different picture of clothing using [Fashion MNIST Dataset](https://github.com/zalandoresearch/fashion-mnist).

In [1]:
def train_pytorch_model(parameters):
    import logging
    import os

    import torch
    import torch.distributed as dist
    import torch.nn.functional as F
    from torch import nn
    from torch.utils.data import DistributedSampler
    from torchvision import datasets, transforms

    logging.basicConfig(
        format="%(asctime)s %(levelname)-8s %(message)s",
        datefmt="%Y-%m-%dT%H:%M:%SZ",
        level=logging.INFO,
    )

    # Create PyTorch CNN Model.
    class Net(nn.Module):
        def __init__(self):
            super(Net, self).__init__()
            self.conv1 = nn.Conv2d(1, 20, 5, 1)
            self.conv2 = nn.Conv2d(20, 50, 5, 1)
            self.fc1 = nn.Linear(4 * 4 * 50, 500)
            self.fc2 = nn.Linear(500, 10)

        def forward(self, x):
            x = F.relu(self.conv1(x))
            x = F.max_pool2d(x, 2, 2)
            x = F.relu(self.conv2(x))
            x = F.max_pool2d(x, 2, 2)
            x = x.view(-1, 4 * 4 * 50)
            x = F.relu(self.fc1(x))
            x = self.fc2(x)
            return F.log_softmax(x, dim=1)

    # IF GPU is available, nccl dist backend is used. Otherwise, gloo dist backend is used.
    if torch.cuda.is_available():
        device = "cuda"
        backend = "nccl"
    else:
        device = "cpu"
        backend = "gloo"
    
    logging.info(f"Using Device: {device}, Backend: {backend}")

    # Setup PyTorch DDP. Distributed environment will be set automatically by Training Operator.
    dist.init_process_group(backend=backend)
    Distributor = torch.nn.parallel.DistributedDataParallel
    local_rank = int(os.getenv("LOCAL_RANK", 0))
    logging.info(
        "Distributed Training for WORLD_SIZE: {}, RANK: {}, LOCAL_RANK: {}".format(
            dist.get_world_size(),
            dist.get_rank(),
            local_rank,
        )
    )

    # Attach model to the correct device.
    device = torch.device(f"{device}:{local_rank}")
    model = Net().to(device)
    model = Distributor(model)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

    # Get Fashion MNIST Dataset.
    dataset = datasets.FashionMNIST(
        "./data",
        train=True,
        download=True,
        transform=transforms.Compose([transforms.ToTensor()]),
    )

    # Every PyTorchJob worker gets distributed sampler of dataset.
    train_loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=128,
        sampler=DistributedSampler(dataset),
    )

    # Start Training.
    logging.info(f"Start training for RANK: {dist.get_rank()}. WORLD_SIZE: {dist.get_world_size()}")

    for epoch in range(int(parameters["NUM_EPOCHS"])):
        model.train()

        for batch_idx, (data, target) in enumerate(train_loader):
            # Attach tensors to the device.
            data = data.to(device)
            target = target.to(device)

            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()
            if batch_idx % 10 == 0 and dist.get_rank() == 0:
                logging.info(
                    "Train Epoch: {} [{}/{} ({:.0f}%)]\tloss={:.4f}".format(
                        epoch,
                        batch_idx * len(data),
                        len(train_loader.dataset),
                        100.0 * batch_idx / len(train_loader),
                        loss.item(),
                    )
                )
    if dist.get_rank() == 0:
        logging.info("Training is finished")

## Run Training Locally in the Notebook

We are going to download Fashion MNIST Dataset and start local training.

In [3]:
# Set dist env variables to run the above training locally on the Notebook.
import os

os.environ["RANK"] = "0"
os.environ["LOCAL_RANK"] = "0"
os.environ["WORLD_SIZE"] = "1"
os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "1234"

# Train Model locally in the Notebook.
train_pytorch_model({"NUM_EPOCHS": "1"})

2024-10-08T13:58:29Z INFO     Using Device: cpu, Backend: gloo
2024-10-08T13:58:29Z INFO     Distributed Training for WORLD_SIZE: 1, RANK: 0, LOCAL_RANK: 0


100%|██████████| 26421880/26421880 [00:02<00:00, 9155631.80it/s] 


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 1364085.84it/s]

Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 8802674.51it/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 8424610.61it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw




2024-10-08T13:58:33Z INFO     Start training for RANK: 0. WORLD_SIZE: 1
2024-10-08T13:58:33Z INFO     Train Epoch: 0 [0/60000 (0%)]	loss=2.3017
2024-10-08T13:58:33Z INFO     Train Epoch: 0 [1280/60000 (2%)]	loss=2.2850
2024-10-08T13:58:34Z INFO     Train Epoch: 0 [2560/60000 (4%)]	loss=2.2844
2024-10-08T13:58:34Z INFO     Train Epoch: 0 [3840/60000 (6%)]	loss=2.2718
2024-10-08T13:58:34Z INFO     Train Epoch: 0 [5120/60000 (9%)]	loss=2.2489
2024-10-08T13:58:35Z INFO     Train Epoch: 0 [6400/60000 (11%)]	loss=2.2405
2024-10-08T13:58:35Z INFO     Train Epoch: 0 [7680/60000 (13%)]	loss=2.2178
2024-10-08T13:58:35Z INFO     Train Epoch: 0 [8960/60000 (15%)]	loss=2.1755
2024-10-08T13:58:35Z INFO     Train Epoch: 0 [10240/60000 (17%)]	loss=2.1326
2024-10-08T13:58:36Z INFO     Train Epoch: 0 [11520/60000 (19%)]	loss=2.0784
2024-10-08T13:58:36Z INFO     Train Epoch: 0 [12800/60000 (21%)]	loss=1.9585
2024-10-08T13:58:36Z INFO     Train Epoch: 0 [14080/60000 (23%)]	loss=1.8107
2024-10-08T13:58:36

## Start Distributive Training with PyTorchJob

Before creating PyTorchJob, you have to create `TrainingClient()`. It uses [Kubernetes Python client](https://github.com/kubernetes-client/python) to communicate with Kubernetes API server. You can set path and context for [the kubeconfig file](https://kubernetes.io/docs/concepts/configuration/organize-cluster-access-kubeconfig/). The default location for the kubeconfig is `~/.kube/config`.

Kubeflow Training Operator automatically set the appropriate env variables (`MASTER_PORT`, `MASTER_ADDR`, `WORLD_SIZE`, `RANK`) for each PyTorchJob container.

PyTorchJob will train model on 3 epochs with 3 PyTorch workers.

In [2]:
from kubeflow.training import TrainingClient, constants

# Start PyTorchJob Training.
pytorchjob_name = "train-pytorch"

# Since we set `job_kind = PyTorchJob` APIs are going to use PyTorchJob as a default Job kind.
training_client = TrainingClient(job_kind=constants.PYTORCHJOB_KIND)

training_client.create_job(
    name=pytorchjob_name,
    train_func=train_pytorch_model,
    parameters={"NUM_EPOCHS": "3"}, # Input parameters for the train function.
    num_workers=2,  # How many PyTorch Nodes will be created.
    num_procs_per_worker=2, # How many procs per node will be used (e.g. number of CPUs/GPUs in a single Node)
    resources_per_worker={"cpu": "2"}
)

### Check the PyTorchJob Status

Use `TrainingClient()` APIs to get information about created PyTorchJob.

In [3]:
print(f"PyTorchJob Status: {training_client.is_job_running(name=pytorchjob_name)}")

PyTorchJob Status: False


### Get PyTorchJob Pod Names

Since we used 3 workers, PyTorchJob will create 1 master pod and 2 worker pods to execute distributed training.

In [4]:
training_client.get_job_pod_names(pytorchjob_name)

['train-pytorch-master-0', 'train-pytorch-worker-0']

### Get PyTorchJob Training Logs

We can get the logs from the master pod.

Every worker processes 20000 data samples on each epoch since we distribute 60000 samples across 3 workers.

In [5]:
logs, _ = training_client.get_job_logs(pytorchjob_name)

print(logs["train-pytorch-master-0"])

[2024-10-08 13:25:08,740] torch.distributed.run: [WARNING] master_addr is only used for static rdzv_backend and when rdzv_endpoint is not specified.
[2024-10-08 13:25:08,741] torch.distributed.run: [WARNING] 
[2024-10-08 13:25:08,741] torch.distributed.run: [WARNING] *****************************************
[2024-10-08 13:25:08,741] torch.distributed.run: [WARNING] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
[2024-10-08 13:25:08,741] torch.distributed.run: [WARNING] *****************************************
[W socket.cpp:663] [c10d] The IPv6 network addresses of (train-pytorch-worker-0, 23456) cannot be retrieved (gai error: -2 - Name or service not known).
[W socket.cpp:663] [c10d] The IPv6 network addresses of (train-pytorch-worker-0, 23456) cannot be retrieved (gai error: -2 - Name or service not known).
[W socket.cpp:

## Delete PyTorchJob

When PyTorchJob is finished, you can delete the resource.

In [6]:
training_client.delete_job(pytorchjob_name)